In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
from glasbey import create_palette

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

s_palette = sns.cubehelix_palette(as_cmap=True)
pal = sns.color_palette("dark")
cpal = sns.cubehelix_palette(start=-0.25, rot=2, as_cmap=True)
g_palette = create_palette(1024)

In [3]:
from src.utils.tracklets import import_tracklets

roots = ["embryo007", "embryo008", "embryo014a", "embryo016", "embryo018"]
datapath = Path().cwd().parent.parent / "data" / "interim" / "confocal"
plotpath = datapath / "plots"
plotpath.mkdir(exist_ok=True)

spots, tracklets, metadatas, tracklets_joined = import_tracklets(datapath, roots)

In [8]:
print(tracklets_joined.columns)

Index(['start_time', 'end_time', 'start_frame', 'end_frame', 'length',
       'source_spot', 'sink_spot', 'mean_ap_position', 'source_ap_position',
       'sink_ap_position', 'initial_x', 'initial_y', 'final_x', 'final_y',
       'initial_x_um', 'initial_y_um', 'final_x_um', 'final_y_um', 'track_id',
       'mean_edge_distance', 'track_n_tracklets', 'cycle', 'embryo',
       'tracklet_id', 'parent_tracklet', 'n_children', 'e_parent_id'],
      dtype='object')


In [11]:
division_angles = {}

for root in roots:    
    division_angles[root] = pd.DataFrame()
    
    for cycle in range(11, 15):
        # get tracklets for this cycle
        r_tracklets = tracklets[root]
        this_tracklets = r_tracklets[r_tracklets["cycle"] == cycle]
        
        # for each parent that exists, initialize a list of child positions
        source_pos = {tracklet: [] for tracklet in this_tracklets[~this_tracklets["parent_tracklet"].isin([-1, 0])]["parent_tracklet"]}
        children = {tracklet: [] for tracklet in source_pos.keys()}
        
        # assign each tracklets source position to its parent
        x, y = this_tracklets.initial_x_um, this_tracklets.initial_y_um
        for parent, this_x, this_y, id_ in zip(this_tracklets["parent_tracklet"], x, y, this_tracklets["tracklet_id"]):
            if (parent == -1) or (parent == 0):
                continue
            source_pos[parent].append((this_x, this_y))
            children[parent].append(id_)
        
        # remove parents that have the wrong number of children
        source_pos = {tracklet: pos for tracklet, pos in source_pos.items() if len(pos) == 2}
        
        # find all tracklets in previous cycle, and remove the nonparents        
        prev_tracklets = r_tracklets[r_tracklets["cycle"] == cycle - 1]
        prev_tracklets = prev_tracklets[prev_tracklets["tracklet_id"].isin(source_pos.keys())]
            
        missing = [tracklet for tracklet in source_pos.keys() if tracklet not in prev_tracklets["tracklet_id"]]
        source_pos = {tracklet: pos for tracklet, pos in source_pos.items() if tracklet not in missing}
        children = {tracklet: children[tracklet] for tracklet in source_pos.keys()}
        
        x = [prev_tracklets.loc[tracklet, "final_x_um"] for tracklet in source_pos.keys()]
        y = [prev_tracklets.loc[tracklet, "final_y_um"] for tracklet in source_pos.keys()]
        t = [prev_tracklets.loc[tracklet, "end_time"] for tracklet in source_pos.keys()]
        cycle = [prev_tracklets.loc[tracklet, "cycle"] for tracklet in source_pos.keys()]
        id_ = [prev_tracklets.loc[tracklet, "tracklet_id"] for tracklet in source_pos.keys()]
        child1 = [children[tracklet][0] for tracklet in source_pos.keys()]
        child2 = [children[tracklet][1] for tracklet in source_pos.keys()]
        child1_x = [r_tracklets.loc[child, "initial_x_um"] for child in child1]
        child1_y = [r_tracklets.loc[child, "initial_y_um"] for child in child1]
        child2_x = [r_tracklets.loc[child, "initial_x_um"] for child in child2]
        child2_y = [r_tracklets.loc[child, "initial_y_um"] for child in child2]
        track_id = [r_tracklets.loc[tracklet, "track_id"] for tracklet in source_pos.keys()]
        
        div_angle = lambda pt1, pt2: np.arctan2(pt2[1] - pt1[1], pt2[0] - pt1[0])
        angles = [div_angle(pts[0], pts[1]) for pts in source_pos.values()]
        u = np.cos(angles)
        v = np.sin(angles)
        
        df = pd.DataFrame({
            "tracklet_id": id_,
            "track_id": track_id,
            "cycle": cycle,
            "x": x,
            "y": y,
            "t": t,
            "child1": child1,
            "child2": child2,
            "child1_x": child1_x,
            "child1_y": child1_y,
            "child2_x": child2_x,
            "child2_y": child2_y,
            "angle": angles,
            "u": u,
            "v": v,
        })
        
        division_angles[root] = pd.concat([division_angles[root], df])
    division_angles[root].to_csv(datapath / f"{root}_division_angles.csv", index=False)